In [34]:
#已知
U_p = 5.0  # m/s
V_p = 1.5*10**(-5)  # m^2/s

U_lb = 0.05  # 不可壓縮假設
L_lb = 200 # mask 計算 建築物最大邊長

Re = 2000 # 目標雷諾數

# 計算晶格黏滯係數
V_lb = (U_lb * L_lb) / Re  
print(f"計算得到的晶格速度 V_lb        = {V_lb:.4f}")
tau = 3 * V_lb + 0.5
print(f"計算得到的晶格黏滯係數 nu      = {tau:.4f}")
if tau < 0.51:
    print("鬆弛時間<0.51，請調整參數以確保模擬穩定。")

# 計算轉換比例
C_u = U_p / U_lb
print(f"速度轉換比例 C_u              = {C_u:.4f}")

# 計算空間解析度
delta_x = V_p / (V_lb * C_u)
print(f"計算得到的空間解析度 delta_x   = {delta_x:.6f} m")

# 計算時間步長
delta_t = delta_x / C_u
print(f"計算得到的時間步長 delta_t     = {delta_t:.8f} s") 

L_p = L_lb * delta_x
print(f"計算得到的物理空間邊長 L_p    = {L_p:.4f} m")

計算得到的晶格速度 V_lb        = 0.0050
計算得到的晶格黏滯係數 nu      = 0.5150
速度轉換比例 C_u              = 100.0000
計算得到的空間解析度 delta_x   = 0.000030 m
計算得到的時間步長 delta_t     = 0.00000030 s
計算得到的物理空間邊長 L_p    = 0.0060 m


2. LBM 參數比例關係大整理假設你的物理環境（$\nu_p, U_p$）是固定的，當你調整晶格參數時，會產生以下連鎖反應：A. 正比關係 (Proportional)$\tau$ (穩定性) $\propto U_{lb}$：增加格子入口速度，鬆弛時間會變大，模擬會更穩定（但不宜超過 $0.1$）。$\tau$ (穩定性) $\propto L_{lb}$：如果你把建築物畫大一點（像素增加），穩定性也會提高。$\Delta x$ (解析度) $\propto \nu_{p}$：流體的物理黏度越高，在同樣的晶格設定下，每個像素代表的物理距離就越大。$\Delta t$ (時間間隔) $\propto \Delta x$：空間切得越粗，時間步長也會跟著變大。B. 反比關係 (Inversely Proportional)$\tau$ (穩定性) $\propto 1/Re$：這是最頭痛的。雷諾數越高，$\tau$ 越接近 $0.5$，模擬越容易崩潰（NaN）。這就是為什麼高 $Re$ 模擬需要極細的網格或湍流模型。$\Delta x$ (解析度) $\propto 1/U_{p}$：如果你想模擬更快的風速（$U_p$ 變大），在晶格參數不變的情況下，你的 $\Delta x$ 會被迫縮小（代表你需要更多網格才能涵蓋同樣範圍）。$\Delta t$ (時間間隔) $\propto 1/C_u$：速度比例越大，時間解析度越細（每秒要跑更多步）。

In [41]:
def calculate_lbm_parameters(Re_target, U_p, V_p=1.5e-5, L_lb=200, U_lb=0.05):
    """
    計算 LBM 模擬參數
    Re_target: 目標物理雷諾數
    U_p: 物理流速 (m/s)
    V_p: 物理運動黏度 (m^2/s), 預設空氣
    L_lb: 晶格特徵長度 (px)
    U_lb: 晶格入口速度 (lu)
    """
    
    print(f"--- 處理目標 Re: {Re_target} ---")
    
    # 1. 判斷是否需要降階模擬 (Scaling Strategy)
    is_scaled = False
    Re_sim = Re_target
    
    if Re_target > 4000:
        print(f"[降階提醒] 目標 Re={Re_target} 過高，模擬將以降階 Re=4000 執行。")
        Re_sim = 4000
        is_scaled = True
    else:
        print(f"[常規模擬] 使用原始 Re={Re_target} 執行。")

    # 2. 計算晶格黏度 V_lb 與 鬆弛時間 tau
    V_lb = (U_lb * L_lb) / Re_sim
    tau = 3 * V_lb + 0.5
    
    # 3. 計算物理與晶格的轉換比例 (基於相似性)
    # 核心公式: Re = (U_p * L_p) / V_p = (U_lb * L_lb) / V_lb
    # 我們假設物理上的 L_p 尺度是根據 Re_target 定義的
    
    # 速度比例
    C_u = U_p / U_lb
    
    # 空間步長 delta_x (由 V_p = V_lb * delta_x * C_u 反推)
    # 注意：這裡使用 V_lb(模擬) 來推導，確保動力相似性
    delta_x = V_p / (V_lb * C_u)
    
    # 時間步長 delta_t
    delta_t = delta_x / C_u
    
    # 物理特徵長度 L_p
    L_p = L_lb * delta_x

    # 輸出結果
    print(f"  > 模擬用晶格黏度 V_lb : {V_lb:.6f}")
    print(f"  > 鬆弛時間 tau (穩定性): {tau:.4f} " + ("(良好)" if tau > 0.51 else "(危險)"))
    print(f"  > 空間解析度 dx       : {delta_x:.8f} m")
    print(f"  > 時間步長 dt         : {delta_t:.8e} s")
    print(f"  > 對應物理尺寸 Lp     : {L_p:.4f} m")
    
    if is_scaled:
        # Scale Back 邏輯
        scale_factor = Re_target / Re_sim
        print(f"  > [Scale Back] 真實物理黏度比模擬強效約 {scale_factor:.1f} 倍")
        print(f"    (註：後處理時，速度分佈比率不變，但擴散效應需按此比例修正)")
    
    return {
        "tau": tau,
        "dx": delta_x,
        "dt": delta_t,
        "L_p": L_p
    }

# --- 測試範例 ---
# 1. 常規範圍 (Re=1000)
calculate_lbm_parameters(Re_target=1000, U_p=5.0, L_lb=200)

print("\n" + "="*40 + "\n")
calculate_lbm_parameters(Re_target=10000, U_p=20.0, L_lb=200)

print("\n" + "="*40 + "\n")

# 2. 高雷諾數降階 (Re=10000)
calculate_lbm_parameters(Re_target=1000, U_p=5.0, L_lb=200)
print("\n" + "="*40 + "\n")
calculate_lbm_parameters(Re_target=10000, U_p=20.0, L_lb=200)

--- 處理目標 Re: 1000 ---
[常規模擬] 使用原始 Re=1000 執行。
  > 模擬用晶格黏度 V_lb : 0.010000
  > 鬆弛時間 tau (穩定性): 0.5300 (良好)
  > 空間解析度 dx       : 0.00001500 m
  > 時間步長 dt         : 1.50000000e-07 s
  > 對應物理尺寸 Lp     : 0.0030 m


--- 處理目標 Re: 10000 ---
[降階提醒] 目標 Re=10000 過高，模擬將以降階 Re=4000 執行。
  > 模擬用晶格黏度 V_lb : 0.002500
  > 鬆弛時間 tau (穩定性): 0.5075 (危險)
  > 空間解析度 dx       : 0.00001500 m
  > 時間步長 dt         : 3.75000000e-08 s
  > 對應物理尺寸 Lp     : 0.0030 m
  > [Scale Back] 真實物理黏度比模擬強效約 2.5 倍
    (註：後處理時，速度分佈比率不變，但擴散效應需按此比例修正)


--- 處理目標 Re: 1000 ---
[常規模擬] 使用原始 Re=1000 執行。
  > 模擬用晶格黏度 V_lb : 0.010000
  > 鬆弛時間 tau (穩定性): 0.5300 (良好)
  > 空間解析度 dx       : 0.00001500 m
  > 時間步長 dt         : 1.50000000e-07 s
  > 對應物理尺寸 Lp     : 0.0030 m


--- 處理目標 Re: 10000 ---
[降階提醒] 目標 Re=10000 過高，模擬將以降階 Re=4000 執行。
  > 模擬用晶格黏度 V_lb : 0.002500
  > 鬆弛時間 tau (穩定性): 0.5075 (危險)
  > 空間解析度 dx       : 0.00001500 m
  > 時間步長 dt         : 3.75000000e-08 s
  > 對應物理尺寸 Lp     : 0.0030 m
  > [Scale Back] 真實物理黏度比模擬強效約 2.5 倍
    (註：後處理時，速度分佈比率不

{'tau': 0.5075, 'dx': 1.5e-05, 'dt': 3.75e-08, 'L_p': 0.003}